In [1]:
import pandas as pd
import numpy as np
import hdbscan
from bertopic import BERTopic

2021-08-04 14:54:47.779605: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Test 1

In [3]:
def get_topic_documents(cluster_id, condensed_tree):
        result_points = np.array([])
        result_points_val = np.array([])
        
        #assert cluster_id > -1, "The topic's label should be greater than -1!"
        
        if cluster_id <= -1:
            return result_points.astype(np.int64), result_points_val.astype(np.float64)
            
        raw_tree = condensed_tree._raw_tree
        
        # Just the cluster elements of the tree, excluding singleton points
        cluster_tree = raw_tree[raw_tree['child_size'] > 1]
        
        # Get the leaf cluster nodes under the cluster we are considering
        leaves = hdbscan.plots._recurse_leaf_dfs(cluster_tree, cluster_id)
        
        # Now collect up the last remaining points of each leaf cluster (the heart of the leaf) 
        for leaf in leaves:
            #max_lambda = raw_tree['lambda_val'][raw_tree['parent'] == leaf].max()
            #points = raw_tree['child'][(raw_tree['parent'] == leaf) & (raw_tree['lambda_val'] == max_lambda)]
            #points_val = raw_tree['lambda_val'][(raw_tree['parent'] == leaf) & (raw_tree['lambda_val'] == max_lambda)]
            points = raw_tree['child'][(raw_tree['parent'] == leaf)]
            points_val = raw_tree['lambda_val'][(raw_tree['parent'] == leaf)]
            result_points = np.hstack((result_points, points))
            result_points_val = np.hstack((result_points_val, points_val))   
        return result_points.astype(np.int64), result_points_val.astype(np.float64)

def generate_topic_documents(bert_model):
    clusterer = bert_model.hdbscan_model
    tree = clusterer.condensed_tree_
    clusters = tree._select_clusters()

    number_of_topics = len(clusters)

    relevant_columns = ['topic', 'document', 'lambda_val']
    df_rel_docs = pd.DataFrame(columns=relevant_columns)

    for i in range(0, number_of_topics):
        rel_docs, lambda_vals = get_topic_documents(clusters[i], tree)
        topic_name = bert_model.topic_names[i]
        for j in range(0, len(rel_docs)):
            new_doc_rel = {}
            new_doc_rel['topic'] = topic_name
            new_doc_rel['document'] = rel_docs[j]
            new_doc_rel['lambda_val'] = round(lambda_vals[j],6)
            df_rel_docs = df_rel_docs.append(new_doc_rel, ignore_index=True)
    del bert_model
    return df_rel_docs

In [3]:
def generate_topic_documents_hdbscan(bert_model):
        clusterer = bert_model.hdbscan_model

        doc_topic_columns = ['document', 'topic', 'probabilities']
        df_doc_topic = pd.DataFrame(columns=doc_topic_columns)

        for i, _ in enumerate(clusterer.labels_):
            new_doc_topic = {}
            new_doc_topic['document'] = i
            new_doc_topic['topic'] = clusterer.labels_[i]
            new_doc_topic['probabilities'] = clusterer.probabilities_[i]
            df_doc_topic = df_doc_topic.append(new_doc_topic, ignore_index=True)
        del bert_model
        return df_doc_topic

In [4]:
topic_model = BERTopic.load('../raw_data/proj_final/50_docs_per_topic/2016_12_BERTopic_model_2_2_raw_content')

In [5]:
df = generate_topic_documents_hdbscan(topic_model)

In [6]:
df.groupby(by=['topic']).count()

,document,probabilities
topic,,
-1.0,1374,1374
0.0,127,127
1.0,118,118
2.0,173,173
3.0,123,123
4.0,74,74
5.0,60,60
6.0,118,118
7.0,73,73


In [67]:
df1 = generate_topic_documents(topic_model)

In [69]:
#df1.groupby(by=['topic']).count()
df1.shape

,topic,document,lambda_val
0,0_of the_in the_to the_and the,1172,1.458804
1,0_of the_in the_to the_and the,3495,1.568285
2,0_of the_in the_to the_and the,1418,1.602545
3,0_of the_in the_to the_and the,4008,1.605422
4,0_of the_in the_to the_and the,1314,1.640275
...,...,...,...
3343,20_the company_san francisco_the car_the model,889,2.242479
3344,20_the company_san francisco_the car_the model,536,2.242479
3345,20_the company_san francisco_the car_the model,4315,2.242479
3346,20_the company_san francisco_the car_the model,3265,2.242479


In [9]:
#topic_model = BERTopic.load('../raw_data/proj_final/50_docs_per_topic/2016_12_BERTopic_model_2_2_raw_content')
#topic_model = BERTopic.load('../raw_data/proj_final/50_docs_per_topic/2016_9_BERTopic_model_2_2_raw_content')
clusterer = topic_model.hdbscan_model
tree = clusterer.condensed_tree_
clusters = tree._select_clusters()

print(topic_model.topic_names[-1])
print(len(clusterer.labels_[clusterer.labels_==-1]))
print(topic_model.topic_names[0])
print(len(clusterer.labels_[clusterer.labels_==0]))
print(topic_model.topic_names[1])
print(len(clusterer.labels_[clusterer.labels_==1]))

-1_in the_to the_on the_to be
1374
0_of the_in the_to the_and the
127
1_the russian_the cia_the russians_that russia
118


In [10]:
len(clusterer.labels_)

5876

In [143]:
np.unique(clusterer.labels_)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20])

In [148]:
clusterer

bool

In [139]:
treexxxxx = clusterer.condensed_tree_.to_pandas()
cluster_treexxxx = treexxxxx[treexxxxx.child_size > 1]

In [144]:
cluster_treexxxx

,parent,child,lambda_val,child_size
105,5876,5877,0.658587,127
106,5876,5878,0.658587,5644
124,5878,5879,0.761768,118
125,5878,5880,0.761768,5518
458,5880,5881,0.910968,173
459,5880,5882,0.910968,5249
507,5882,5883,0.919004,123
508,5882,5884,0.919004,5103
559,5884,5885,0.936853,74
560,5884,5886,0.936853,5013


In [12]:
print(len(np.unique(clusterer.labels_)))
print(len(clusters))
print(topic_model.topic_names[1])

22
21
1_the russian_the cia_the russians_that russia


In [21]:
np.unique(clusters)

array([5877., 5879., 5881., 5883., 5885., 5887., 5889., 5891., 5894.,
       5895., 5899., 5901., 5903., 5906., 5907., 5909., 5910., 5911.,
       5915., 5917., 5918.])

In [129]:
df_dataset = pd.read_csv('../raw_data/proj_final/50_docs_per_topic/2016_12_dataset.csv')
print(len(clusterer.labels_))
print(df_dataset.shape)

5876
(5876, 7)


In [137]:
clusterer.prediction_data_

In [130]:
df[df['topic']==13]

,document,topic,probabilities
133,133.0,13.0,0.750616
145,145.0,13.0,0.991617
151,151.0,13.0,1.000000
159,159.0,13.0,0.917316
160,160.0,13.0,1.000000
...,...,...,...
5674,5674.0,13.0,1.000000
5732,5732.0,13.0,0.809462
5737,5737.0,13.0,0.612206
5809,5809.0,13.0,0.348455


In [158]:
print(clusterer.labels_[0])
topic_model.topic_names

10


{-1: '-1_in the_to the_on the_to be',
 0: '0_of the_in the_to the_and the',
 1: '1_the russian_the cia_the russians_that russia',
 2: '2_in the_the attack_islamic state_he was',
 3: '3_security council_the united_the un_the palestinians',
 4: '4_the syrian_in aleppo_of aleppo_in syria',
 5: '5_the trump_mr trump_conflicts of_trump organization',
 6: '6_the fed_the economy_interest rates_the dollar',
 7: '7_planned parenthood_she was_women who_that she',
 8: '8_climate change_the epa_energy department_transition team',
 9: '9_the united_united states_to mexico_donald trump',
 10: '10_the european_prime minister_european union_the eu',
 11: '11_one china_the chinese_of taiwan_with taiwan',
 12: '12_air force_china sea_the plane_force one',
 13: '13_the team_the nfl_the giants_week 15',
 14: '14_the military_secretary of_general mattis_donald trump',
 15: '15_secretary of_rex tillerson_exxon mobil_tillerson has',
 16: '16_affordable care_care act_health insurance_health care',
 17: '17_th

In [169]:
df_dataset[df_dataset['topic']==8]

,Unnamed: 0,Unnamed: 0.1,id,title,year,month,content,topic,doc_id
19,19,81463,20599,"Jo Cox, Member of British Parliament, Is Kille...",2016.0,12.0,"LEEDS, England — A member of Parliament was...",8,19
32,32,81477,20676,Britain’s Dreams of a ‘Swiss Miracle’ Look Mor...,2016.0,12.0,To help explain why the British voted to leave...,8,32
42,42,81490,20754,A Blunt Message After ‘Brexit’: Bolting Will C...,2016.0,12.0,BRUSSELS — Shaken by Britain’s vote to quit...,8,42
43,43,81491,20755,"Having Won, Boris Johnson and ‘Brexit’ Leaders...",2016.0,12.0,LONDON — With their giddy celebrations of “...,8,43
91,91,81548,22635,"Italy’s Premier, Matteo Renzi, Says He’ll Resi...",2016.0,12.0,ROME — Italy plunged into political and eco...,8,91
...,...,...,...,...,...,...,...,...,...
5405,5405,88982,217368,"Using similar tactics, Austrian nationalists h...",2016.0,12.0,VIENNA — He is a political disrupter supp...,8,5405
5428,5428,89009,217409,Italian prime minister resigns in populist revolt,2016.0,12.0,BRUSSELS — Europe’s embattled political...,8,5428
5456,5456,89039,217447,"Anti-immigrant, anti-euro populists gain groun...",2016.0,12.0,ROME — Italy’s parties vowed Monday to ...,8,5456
5531,5531,89122,217548,Anti-Islam Dutch politician Geert Wilders foun...,2016.0,12.0,LONDON — A Dutch court found politician...,8,5531


In [166]:
#docs_df = pd.DataFrame(data, columns=["Doc"])
#df_dataset['topic'] = clusterer.labels_
#df_dataset['doc_id'] = range(len(df_dataset['content']))
#docs_per_topic = df_dataset.groupby(['topic'], as_index = False).agg({'content': ' '.join})
#docs_per_topic
df_dataset

,Unnamed: 0,Unnamed: 0.1,id,title,year,month,content,topic,doc_id
0,0,81444,17283,House Republicans Fret About Winning Their Hea...,2016.0,12.0,WASHINGTON — Congressional Republicans have...,10,0
1,1,81445,17291,"First, a Mixtape. Then a Romance. - The New Yo...",2016.0,12.0,"Just how is Hillary Kerr, the founder of ...",20,1
2,2,81446,17292,Calling on Angels While Enduring the Trials of...,2016.0,12.0,Angels are everywhere in the Muñiz family’s ap...,20,2
3,3,81447,20127,U.S. Plans to Step Up Military Campaign Agains...,2016.0,12.0,"ABU DHABI, United Arab Emirates — The Obama...",-1,3
4,4,81448,20131,272 Slaves Were Sold to Save Georgetown. What ...,2016.0,12.0,WASHINGTON — The human cargo was loaded on ...,20,4
...,...,...,...,...,...,...,...,...,...
5871,5871,89520,218078,An eavesdropping Uber driver saved his 16-year...,2016.0,12.0,Uber driver Keith Avila picked up a p...,20,5871
5872,5872,89521,218079,Plane carrying six people returning from a Cav...,2016.0,12.0,Crews on Friday continued to search L...,1,5872
5873,5873,89522,218080,After helping a fraction of homeowners expecte...,2016.0,12.0,When the Obama administration announced a...,10,5873
5874,5874,89523,218081,"Yes, this is real: Michigan just banned bannin...",2016.0,12.0,This story has been updated. A new law in...,19,5874


In [154]:
topic_model.topic_names[clusterer.labels_[0]]

'9_the united_united states_to mexico_donald trump'

In [53]:
df_dataset['content'].iloc[0]

'WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been 

In [39]:
xx = np.unique(clusterer.labels_)
for x in xx:
    print(topic_model.topic_names[x])

-1_in the_to the_on the_to be
0_of the_in the_to the_and the
1_the russian_the cia_the russians_that russia
2_in the_the attack_islamic state_he was
3_security council_the united_the un_the palestinians
4_the syrian_in aleppo_of aleppo_in syria
5_the trump_mr trump_conflicts of_trump organization
6_the fed_the economy_interest rates_the dollar
7_planned parenthood_she was_women who_that she
8_climate change_the epa_energy department_transition team
9_the united_united states_to mexico_donald trump
10_the european_prime minister_european union_the eu
11_one china_the chinese_of taiwan_with taiwan
12_air force_china sea_the plane_force one
13_the team_the nfl_the giants_week 15
14_the military_secretary of_general mattis_donald trump
15_secretary of_rex tillerson_exxon mobil_tillerson has
16_affordable care_care act_health insurance_health care
17_the pipeline_of the_north dakota_rock sioux
18_the united_united states_white house_according to
19_north carolina_the state_the governor_the 

In [70]:
for i, _ in enumerate(clusterer.labels_):
    print(df_dataset[['title']].iloc[i])

title    House Republicans Fret About Winning Their Hea...
Name: 0, dtype: object
title    First, a Mixtape. Then a Romance. - The New Yo...
Name: 1, dtype: object
title    Calling on Angels While Enduring the Trials of...
Name: 2, dtype: object
title    U.S. Plans to Step Up Military Campaign Agains...
Name: 3, dtype: object
title    272 Slaves Were Sold to Save Georgetown. What ...
Name: 4, dtype: object
title    Get to Know the Historical Figures on the $5, ...
Name: 5, dtype: object
title    Queen Elizabeth’s Party Planner Is Proud to We...
Name: 6, dtype: object
title    Small Rhode Island Suddenly Has Big Role in Pr...
Name: 7, dtype: object
title    Two-Hour Marathon. Can It Be Done? - The New Y...
Name: 8, dtype: object
title    How Kosovo Was Turned Into Fertile Ground for ...
Name: 9, dtype: object
title    Tony Fadell Steps Down Amid Tumult at Nest, a ...
Name: 10, dtype: object
title    Garbiñe Muguruza Upsets Serena Williams in Fre...
Name: 11, dtype: object
title    Panam

title    Beware, iPhone Users: Fake Retail Apps Are Sur...
Name: 104, dtype: object
title    The Rehabilitation of a Coaching Outcast - The...
Name: 105, dtype: object
title    Pat McCrory, North Carolina Governor, Concedes...
Name: 106, dtype: object
title    Bill Cosby’s Testimony on Drugs Can Be Evidenc...
Name: 107, dtype: object
title    The Hunt for Hatchimals, the Elusive Toy of th...
Name: 108, dtype: object
title    Keep Your Family Close and Your Roomba Closer ...
Name: 109, dtype: object
title    Finding a Home in the Bronx After Fleeing Anti...
Name: 110, dtype: object
title    Bob Dole Worked Behind the Scenes on Trump-Tai...
Name: 111, dtype: object
title    As Fake News Spreads Lies, More Readers Shrug ...
Name: 112, dtype: object
title    Donald Trump, Iran, Joe McKnight: Your Tuesday...
Name: 113, dtype: object
title    Cold Hands May Signal Raynaud’s Phenomenon - T...
Name: 114, dtype: object
title    Saudis Bankroll Taliban, Even as King Official...
Name: 115, dtype:

title    Hillary Clinton’s Team to Join Wisconsin Recou...
Name: 243, dtype: object
title    U.S. Seeks Death for Charleston Shooting Suspe...
Name: 244, dtype: object
title    Officials to Close Standing Rock Protest Camps...
Name: 245, dtype: object
title    More Than Coffee: New York’s Vanishing Diner C...
Name: 246, dtype: object
title    Two Black Women Embrace Their Chance to Be Hoc...
Name: 247, dtype: object
title    Ethics Office Praises Donald Trump for a Move ...
Name: 248, dtype: object
title    Why the Trump Team’s Economic Promises Will Be...
Name: 249, dtype: object
title    Colombia’s Congress Approves Peace Accord With...
Name: 250, dtype: object
title    A Dose of a Hallucinogen From a ‘Magic Mushroo...
Name: 251, dtype: object
title    Italian Monastery Seeks Salvation in Beer Afte...
Name: 252, dtype: object
title    Donald Trump, François Hollande, Tiger Woods: ...
Name: 253, dtype: object
title    Howard Schultz to Step Down as Starbucks Chief...
Name: 254, dtype:

title    Internal Inquiry Sealed the Fate of Roger Aile...
Name: 348, dtype: object
title    Scaling Up a Drug Trade, Straight Through ISIS...
Name: 349, dtype: object
title    Young Rural Women in India Chase Big-City Drea...
Name: 350, dtype: object
title    Where Even Nightmares Are Classified: Psychiat...
Name: 351, dtype: object
title    Angela Merkel to Seek 4th Term as Germany’s Le...
Name: 352, dtype: object
title    Trump Has Options for Undoing Obama’s Climate ...
Name: 353, dtype: object
title    In Havana, Castro’s Death Lays Bare a Generati...
Name: 354, dtype: object
title    At Home With the Macabre - The New York Times
Name: 355, dtype: object
title    Fidel Castro, Cuban Revolutionary Who Defied U...
Name: 356, dtype: object
title    Under Fidel Castro, Sport Symbolized Cuba’s St...
Name: 357, dtype: object
title    Trump Claims, With No Evidence, That ‘Millions...
Name: 358, dtype: object
title    Delta Air Lines Bans Disruptive Donald Trump S...
Name: 359, dtype: obj

title    Zumwalt: Obama’s Russian Hacking Shell Game - ...
Name: 459, dtype: object
title    49ers Give Kaepernick Prestigious Award for ‘I...
Name: 460, dtype: object
title    California Licenses 800K Illegal Aliens as Mot...
Name: 461, dtype: object
title    Washington Post’s Fake News of Russian Vermont...
Name: 462, dtype: object
title    Obama, Democratic Leaders to Plan Last Ditch E...
Name: 463, dtype: object
title    Trump: Putin ’Very Smart’ to Not Retaliate ove...
Name: 464, dtype: object
title    Trump’s Inaugural Parade Lineup: Border Patrol...
Name: 465, dtype: object
title    Brooks: Trump ’Siding With a Foreign Leader Ag...
Name: 466, dtype: object
title    Drudge, Downed by Cyberattack, Suspects Govern...
Name: 467, dtype: object
title    Kellogg’s ‘Values’? Corn Flakes Invented to Cu...
Name: 468, dtype: object
title    Two Texas Guardsmen Killed in Helicopter Crash
Name: 469, dtype: object
title    Portrait of Ronald Reagan Defaced During Break...
Name: 470, dtype: ob

...

title    Berlin and Ankara: a tale of two terror attacks
Name: 5296, dtype: object
title    The 11 best albums you might have missed this ...
Name: 5297, dtype: object
title    Skip Die Hard this year. Black Christmas is th...
Name: 5298, dtype: object
title    Washington, DC, passed one of the nation’s mos...
Name: 5299, dtype: object
title    Electoral integrity in all 50 US states, ranke...
Name: 5300, dtype: object
title    How to use taxes to close the trade deficit
Name: 5301, dtype: object
title    Why are there so few Hanukkah movies?
Name: 5302, dtype: object
title    Obama just took a parting shot at Israel — and...
Name: 5303, dtype: object
title    Patriots Day is a stirring tale of ordinary he...
Name: 5304, dtype: object
title    What the holidays are like for a recovering al...
Name: 5305, dtype: object
title    Hidden Figures, about 3 black women at NASA in...
Name: 5306, dtype: object
title    Sense8 Christmas special review: Netflix’s ser...
Name: 5307, dtype: object


title    Trump, the pleaser president
Name: 5398, dtype: object
title    Donald Trump’s Cabinet assembly instructions
Name: 5399, dtype: object
title    Sorry, Lt. Gen. Flynn, it’s unrealistic to ‘wi...
Name: 5400, dtype: object
title    Romney’s groveling before Trump is horrible to...
Name: 5401, dtype: object
title    After a toddler accidentally shot and killed h...
Name: 5402, dtype: object
title    Washington Monument to remain closed for at le...
Name: 5403, dtype: object
title    One year after the San Bernardino attack, poli...
Name: 5404, dtype: object
title    Using similar tactics, Austrian nationalists h...
Name: 5405, dtype: object
title    Second-chance law for young criminals puts vio...
Name: 5406, dtype: object
title    U.S. economy added 178,000 jobs in November; u...
Name: 5407, dtype: object
title    When Americans land in trouble abroad, these e...
Name: 5408, dtype: object
title    Trump’s emerging Cabinet is looking less Trump...
Name: 5409, dtype: object
title 

title    Annie Glenn: ‘When I called John, he cried. Pe...
Name: 5532, dtype: object
title    A teen fired up Facebook Live from the highway...
Name: 5533, dtype: object
title    A rare half-male, half-female butterfly — and ...
Name: 5534, dtype: object
title    Trump expected to pick ExxonMobil chief Rex Ti...
Name: 5535, dtype: object
title    Who is Rex Tillerson, the ExxonMobil chairman ...
Name: 5536, dtype: object
title    Secret CIA assessment says Russia was trying t...
Name: 5537, dtype: object
title    The CIA concluded that Russia worked to elect ...
Name: 5538, dtype: object
title    Schumer calls for full investigation by Congre...
Name: 5539, dtype: object
title    The best movies of 2016: ‘Moonlight,’ ‘Manches...
Name: 5540, dtype: object
title    ISIS is back in the ancient Syrian city of Pal...
Name: 5541, dtype: object
title    Taiwan is a country with history and people. I...
Name: 5542, dtype: object
title    Five myths about U.S.-China relations
Name: 5543, dtype:

Name: 5640, dtype: object
title    Amid outcry, N.C. GOP’s plan to curb Democrati...
Name: 5641, dtype: object
title    Ivanka Trump could be the most powerful first ...
Name: 5642, dtype: object
title    Trump Grill’s star rating is plummeting, and Y...
Name: 5643, dtype: object
title    After first bitter blast, U.S. faces second wa...
Name: 5644, dtype: object
title    It took three years for Yahoo to tell us about...
Name: 5645, dtype: object
title    Defying skeptics, Kim Jong Un marks five years...
Name: 5646, dtype: object
title    Saddam Hussein should have been left to run Ir...
Name: 5647, dtype: object
title    In last-shot bid, thousands urge electoral col...
Name: 5648, dtype: object
title    Trump is stoking his base on his pre-inaugural...
Name: 5649, dtype: object
title    For a president-elect who touts ‘America first...
Name: 5650, dtype: object
title    Trump names Rep. Mick Mulvaney, a fiscal hawk,...
Name: 5651, dtype: object
title    In Pakistan, five girls were k

title    On Christmas, and ‘taming the savageness of men’
Name: 5766, dtype: object
title    Some lessons from Jesus, for all of us
Name: 5767, dtype: object
title    Evangelicals side with Israel. That’s hurting ...
Name: 5768, dtype: object
title    What the Hanukkah story teaches us about the T...
Name: 5769, dtype: object
title    Europe may face a grim future with terrorism a...
Name: 5770, dtype: object
title    Grace, unwrapped
Name: 5771, dtype: object
title    They’ll be home for Christmas: Five homeless c...
Name: 5772, dtype: object
title    Rockettes aren’t being forced to perform at Tr...
Name: 5773, dtype: object
title    Best and worst moments from the Redskins’ win ...
Name: 5774, dtype: object
title    NFL Week 16: Carr and Mariota suffer broken fi...
Name: 5775, dtype: object
title    5 mistakes Americans are making with their money
Name: 5776, dtype: object
title    5 reasons December is a horrible time to date
Name: 5777, dtype: object
title    9 souvenirs you’d be 

In [75]:
bert_model = topic_model
clusterer = bert_model.hdbscan_model

In [76]:
tree = clusterer.condensed_tree_
clusters = tree._select_clusters()

In [83]:
cluster_id = clusters[0]
condensed_tree = tree
raw_tree = condensed_tree._raw_tree

In [91]:
raw_tree[raw_tree['child_size'] > 1]

array([(5876, 5877, 0.65858744,  127), (5876, 5878, 0.65858744, 5644),
       (5878, 5879, 0.76176836,  118), (5878, 5880, 0.76176836, 5518),
       (5880, 5881, 0.91096753,  173), (5880, 5882, 0.91096753, 5249),
       (5882, 5883, 0.91900434,  123), (5882, 5884, 0.91900434, 5103),
       (5884, 5885, 0.93685279,   74), (5884, 5886, 0.93685279, 5013),
       (5886, 5887, 0.9903625 ,   60), (5886, 5888, 0.9903625 , 4891),
       (5888, 5889, 1.09116835,  118), (5888, 5890, 1.09116835, 4622),
       (5890, 5891, 1.24556222,   73), (5890, 5892, 1.24556222, 4238),
       (5892, 5893, 1.25615491, 4099), (5892, 5894, 1.25615491,  125),
       (5893, 5895, 1.32179313,  426), (5893, 5896, 1.32179313, 3548),
       (5896, 5897, 1.32484042,  742), (5896, 5898, 1.32484042, 2801),
       (5898, 5899, 1.34028009,   77), (5898, 5900, 1.34028009, 2703),
       (5897, 5901, 1.40872835,  145), (5897, 5902, 1.40872835,  561),
       (5902, 5903, 1.54409042,  151), (5902, 5904, 1.54409042,  357),
      

In [82]:
cluster_tree = raw_tree[raw_tree['child_size'] > 1]
cluster_tree

array([(5876, 5877, 0.65858744,  127), (5876, 5878, 0.65858744, 5644),
       (5878, 5879, 0.76176836,  118), (5878, 5880, 0.76176836, 5518),
       (5880, 5881, 0.91096753,  173), (5880, 5882, 0.91096753, 5249),
       (5882, 5883, 0.91900434,  123), (5882, 5884, 0.91900434, 5103),
       (5884, 5885, 0.93685279,   74), (5884, 5886, 0.93685279, 5013),
       (5886, 5887, 0.9903625 ,   60), (5886, 5888, 0.9903625 , 4891),
       (5888, 5889, 1.09116835,  118), (5888, 5890, 1.09116835, 4622),
       (5890, 5891, 1.24556222,   73), (5890, 5892, 1.24556222, 4238),
       (5892, 5893, 1.25615491, 4099), (5892, 5894, 1.25615491,  125),
       (5893, 5895, 1.32179313,  426), (5893, 5896, 1.32179313, 3548),
       (5896, 5897, 1.32484042,  742), (5896, 5898, 1.32484042, 2801),
       (5898, 5899, 1.34028009,   77), (5898, 5900, 1.34028009, 2703),
       (5897, 5901, 1.40872835,  145), (5897, 5902, 1.40872835,  561),
       (5902, 5903, 1.54409042,  151), (5902, 5904, 1.54409042,  357),
      

In [106]:
print(df_dataset.shape)
leaves = hdbscan.plots._recurse_leaf_dfs(cluster_tree, cluster_id)
print(leaves)

(5876, 7)
[5877.0]


In [108]:
len(clusters)

21

In [109]:
for cl in range(0, len(clusters)):
    leaves = hdbscan.plots._recurse_leaf_dfs(cluster_tree, clusters[cl])
    print(leaves)

[5877.0]
[5879.0]
[5881.0]
[5883.0]
[5885.0]
[5887.0]
[5889.0]
[5891.0]
[5894.0]
[5913, 5914]
[5899.0]
[5901.0]
[5903.0]
[5906.0]
[5907.0]
[5909.0]
[5910.0]
[5911.0]
[5915.0]
[5917.0]
[5927, 5928, 5931, 5932, 5925, 5929, 5930, 5922]


In [115]:
points_list = []
for cl in range(0, len(clusters)):
    leaves = hdbscan.plots._recurse_leaf_dfs(cluster_tree, clusters[cl])
    result_points = np.array([])
    result_points_val = np.array([])
    for leaf in leaves:
        #max_lambda = raw_tree['lambda_val'][raw_tree['parent'] == leaf].max()
        #points = raw_tree['child'][(raw_tree['parent'] == leaf) & (raw_tree['lambda_val'] == max_lambda)]
        #points_val = raw_tree['lambda_val'][(raw_tree['parent'] == leaf) & (raw_tree['lambda_val'] == max_lambda)]
        points = raw_tree['child'][(raw_tree['parent'] == leaf)]
        points_val = raw_tree['lambda_val'][(raw_tree['parent'] == leaf)]
        
        result_points = np.hstack((result_points, points))
        result_points_val = np.hstack((result_points_val, points_val))   
        points_list.append(result_points)        

In [122]:
xx = np.array(points_list)
kk = np.array([])
for x in xx:
    kk = np.hstack((kk, x))
kk    

array([1172., 3495., 1418., ..., 4315., 3265., 3376.])

In [170]:
len(np.unique(kk))

3348

In [173]:
topic_model = BERTopic.load('../raw_data/proj_final/10_docs_per_topic/2016_9_BERTopic_model_2_2_raw_content')
df_dataset = pd.read_csv('../raw_data/proj_final/10_docs_per_topic/2016_9_dataset.csv')
df_topic = pd.read_csv('../raw_data/proj_final/10_docs_per_topic/2016_9_BERTopic_Info.csv')
docs = df_dataset['content'].values
topics  = df_topic['Topic'].values
topics, probs = topic_model.fit_transform(docs)

In [177]:
new_topics, new_probs = topic_model.reduce_topics(docs, topics, probabilities=probs, nr_topics=18)
new_topics

IndexError: index 129 is out of bounds for axis 1 with size 129

In [191]:
probs

array([[6.86264621e-003, 3.22731969e-003, 7.14934155e-003, ...,
        1.02438619e-002, 1.00701062e-002, 8.59272550e-003],
       [1.01948770e-003, 1.64285811e-003, 1.06858406e-003, ...,
        2.06660344e-003, 1.99601114e-003, 2.08616176e-003],
       [7.25168866e-004, 9.23610481e-004, 6.87608392e-004, ...,
        1.30461630e-003, 1.34605736e-003, 1.61309061e-003],
       ...,
       [1.72938117e-307, 1.36372769e-307, 2.08835531e-307, ...,
        3.14309238e-307, 3.73655166e-307, 2.68425320e-307],
       [1.72145197e-307, 1.36134629e-307, 2.08603864e-307, ...,
        3.10159120e-307, 3.69930204e-307, 2.66582545e-307],
       [8.67219956e-308, 1.79779451e-307, 9.45775809e-308, ...,
        1.31951923e-307, 1.49547094e-307, 1.55844380e-307]])

In [180]:
df_dataset.index.values

array([   0,    1,    2, ..., 5680, 5681, 5682])

In [187]:
#df_news = pd.DataFrame(data={'doc_id': df_dataset.index.values, 'topic': topics, 'probabilities': probs})
df_news = pd.DataFrame(data={'doc_id': df_dataset.index.values, 'topic': topics})
df_news

,doc_id,topic
0,0,12
1,1,-1
2,2,-1
3,3,20
4,4,-1
...,...,...
5678,5678,-1
5679,5679,100
5680,5680,56
5681,5681,56


In [ ]:
df_news['topic'].min()

## Test 2

In [6]:
#topic_model = BERTopic.load('../raw_data/proj_final/10_docs_per_topic/2016_9_BERTopic_model_2_2_raw_content')
#df_dataset = pd.read_csv('../raw_data/proj_final/10_docs_per_topic/2016_9_dataset.csv')
#df_topic = pd.read_csv('../raw_data/proj_final/10_docs_per_topic/2016_9_BERTopic_Info.csv')
#docs = df_dataset['content'].values
#topics  = df_topic['Topic'].values
#topics, probs = topic_model.fit_transform(docs)

In [6]:
df_topic_docs = pd.read_csv('../raw_data/proj_final/10_docs_per_topic/2016_12_BERTopic_TopicDocuments_reduction.csv')
print(df_topic_docs.shape)
#df_temp = df_topic_documents_hdbscan[df_topic_docs['topic']==-1].sort_values(by=['probabilities'], ascending=True).reset_index(drop=True)
df_temp = df_topic_docs[df_topic_docs['topic']==-1].reset_index(drop=True)

(5876, 2)


In [5]:
df_temp

,document_id,topic
0,2,-1
1,4,-1
2,5,-1
3,6,-1
4,7,-1
...,...,...
1651,5862,-1
1652,5864,-1
1653,5868,-1
1654,5870,-1
